In [7]:
from transformers import pipeline
from datasets import load_dataset
from evaluate import evaluator
import evaluate
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch
from evaluate import visualization
import pandas as pd

In [8]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [12]:
data = load_dataset("navjordj/SNL_summarization", split="validation")
metric = evaluate.load("rouge")


Using custom data configuration navjordj--SNL_summarization-d89e75929608302f
Found cached dataset parquet (C:/Users/jorgen/.cache/huggingface/datasets/navjordj___parquet/navjordj--SNL_summarization-d89e75929608302f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 12.14it/s]


In [13]:
data

DatasetDict({
    validation: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 819
    })
    train: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 10874
    })
    test: Dataset({
        features: ['id', 'url', 'date_scraped', 'headline', 'category', 'ingress', 'article'],
        num_rows: 1300
    })
})

In [5]:
model = T5ForConditionalGeneration.from_pretrained("navjordj/t5-base-snl")
tokenizer = AutoTokenizer.from_pretrained("navjordj/t5-base-snl")

In [33]:
greedy_config = {
    "max_length": 120,
}

beam_config = {
    "max_length": 120,
    "num_beams": 6,
    "early_stopping": True
}

beam_no_stopping_config = {
    "max_length": 120,
    "num_beams": 6
}

beam_no_ngram_2 = {
    "max_length": 120,
    "num_beams": 6,
    "no_repeat_ngram_size": 2, 
    "early_stopping": True
}

beam_no_ngram_4 = {
    "max_length": 120,
    "num_beams": 6,
    "no_repeat_ngram_size": 4, 
    "early_stopping": True
}

beam_no_ngram_12 = {
    "max_length": 120,
    "num_beams": 6,
    "no_repeat_ngram_size": 4, 
    "early_stopping": True
}

sample_config = {
    "max_length": 120,
    "do_sample": True,
    "top_k": 0
}

sample_temperature_config = {
    "max_length": 120,
    "do_sample": True,
    "top_k": 0,
    "temperature": 0.7
}

top_k_config = {
    "max_length": 120,
    "do_sample": True,
    "top_k": 50,
}

top_p_config = {
    "max_length": 120,
    "do_sample": True,
    "top_p": 0.92, 
    "top_k": 0
}

configs = [greedy_config, beam_config, beam_no_stopping_config, beam_no_ngram_2, beam_no_ngram_4, sample_config, sample_temperature_config, top_k_config, top_p_config]

In [35]:
task_evaluator = evaluator("summarization")

results_df = pd.DataFrame(columns=["config", 'rouge1','rouge2', 'rougeL', 'rougeLsum', 'total_time_in_seconds', 'samples_per_second'])

for config in configs[:3]:
    print(config)
    pipe = pipeline("summarization", model=model, tokenizer=tokenizer, batch_size=4, device=device, **config)

    results = task_evaluator.compute(
        model_or_pipeline=pipe,
        data=data,
        input_column="article",
        label_column="ingress",
        metric=metric
    )

    results_df = results_df.append({"config": config, **results}, ignore_index=True)

results_df.to_csv("hyperparam_search_results.csv")

{'max_length': 120}


RuntimeError: CUDA out of memory. Tried to allocate 114.00 MiB (GPU 0; 8.00 GiB total capacity; 7.21 GiB already allocated; 0 bytes free; 7.36 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF